# Setup

In [1]:
import pandas as pd
import pymysql.cursors
from sqlalchemy import create_engine, MetaData
from sqlalchemy.schema import CreateTable
from sqlalchemy import text
import getpass
import os

sql_pass = getpass.getpass()
#sql_pass = os.environ['MySQLPass']
connection_string = 'mysql+pymysql://root:' + sql_pass + '@localhost:3306/'
engine = create_engine(connection_string)


# People

### Names

In [2]:
names= pd.read_csv('profiles/names.csv', sep=';')

In [3]:
names.to_sql('people_names', engine, 'civictech', if_exists='replace', index=False)

47

### Profiles

In [4]:
people=pd.read_csv('profiles/people_main_db.csv', sep=';')
print(list(people.columns))
print(len(list(people.columns)))

['ind_id', 'geoCountryName', 'geoLocationName', 'summary', 'industryName', 'headline', 'experience', 'education', 'languages', 'publications', 'certifications', 'volunteer', 'honors', 'projects', 'experience1', 'experience2', 'experience3', 'experience4', 'experience5', 'education1', 'education2', 'education3', 'languages_over2', 'honors_stated', 'publications_stated', 'volunteer_stated', 'projects_stated', 'certifications_stated', 'languages_stated', 'consulting_roles', 'direction_roles', 'founding_roles']
32


In [5]:
# We may want to drop the columns with too many missing values.
pd.DataFrame(people.isna().sum().sort_values(ascending=False)).head(15)

,0
summary,15
education3,12
education2,6
geoLocationName,4
experience5,3
experience4,2
geoCountryName,2
education1,1
honors_stated,0
languages_over2,0


In [6]:
# and/or drop text columns 
people.dtypes.sort_values()

ind_id                    int64
consulting_roles          int64
languages_stated          int64
certifications_stated     int64
projects_stated           int64
volunteer_stated          int64
publications_stated       int64
honors_stated             int64
languages_over2           int64
direction_roles           int64
founding_roles            int64
volunteer                object
geoCountryName           object
geoLocationName          object
summary                  object
industryName             object
headline                 object
experience               object
education                object
languages                object
certifications           object
education3               object
education1               object
experience5              object
experience4              object
experience3              object
publications             object
experience1              object
projects                 object
honors                   object
education2               object
experien

In [7]:
people.to_csv("sql/people.csv", sep=";", index=False)

In [81]:
# for now we are keeping it as is
people.to_sql('people', engine, 'civictech', if_exists='replace', index=False)

47

### Experience

In [9]:
experience=pd.read_csv('profiles/people_experience_db.csv', sep=';')

In [10]:
print(list(experience.columns))
print(len(list(experience.columns)))
print(len(experience))

['index', 'ind_id', 'locationName', 'companyName', 'description', 'title', 'startDate_month', 'startDate_year', 'endDate_month', 'endDate_year', 'industry', 'company_empl_low', 'company_empl_high', 'title_direction', 'titleconsulting', 'titlefounder']
16
235


In [11]:
experience.isna().sum().sort_values(ascending=False)

locationName         62
description          57
industry             20
companyName           5
title                 5
index                 0
ind_id                0
startDate_month       0
startDate_year        0
endDate_month         0
endDate_year          0
company_empl_low      0
company_empl_high     0
title_direction       0
titleconsulting       0
titlefounder          0
dtype: int64

In [12]:
experience.isna().sum().sort_values(ascending=False)

locationName         62
description          57
industry             20
companyName           5
title                 5
index                 0
ind_id                0
startDate_month       0
startDate_year        0
endDate_month         0
endDate_year          0
company_empl_low      0
company_empl_high     0
title_direction       0
titleconsulting       0
titlefounder          0
dtype: int64

In [13]:
experience['startDate_year'] = experience['startDate_year'].fillna(0)
experience['startDate_month'] = experience['startDate_month'].fillna(0)
experience['endDate_year'] = experience['endDate_year'].fillna(0)
experience['endDate_month'] = experience['endDate_month'].fillna(0)
experience['company_empl_low'] = experience['company_empl_low'].fillna(999)
experience['company_empl_high'] = experience['company_empl_high'].fillna(999)

In [14]:
experience['startDate_year'] = experience['startDate_year'].astype(int)
experience['startDate_month'] = experience['startDate_month'].astype(int)
experience['endDate_year'] = experience['endDate_year'].astype(int)
experience['endDate_month'] = experience['endDate_month'].astype(int)
experience['company_empl_low'] = experience['company_empl_low'].astype(int)
experience['company_empl_high'] = experience['company_empl_high'].astype(int)

In [15]:
experience.industry[1]

"['Management Consulting']"

In [16]:
import re
def clean_industry(x):
    x=str(x)
    pattern = "[a-zA-Z]+ [a-zA-Z]+|[a-zA-Z]+"
    a= re.findall(pattern, x)
    a= ' '.join(a)
    #a= a[1:-1]
    return a

In [17]:
print(list(experience['industry'].apply(clean_industry)))

['Management Consulting', 'Management Consulting', 'Management Consulting', 'Management Consulting', 'Management Consulting', 'Venture Capital Private Equity', 'Venture Capital Private Equity', 'Venture Capital Private Equity', 'Venture Capital Private Equity', 'Venture Capital Private Equity', 'Nonprofit Organization Management', 'Nonprofit Organization Management', 'Nonprofit Organization Management', 'Nonprofit Organization Management', 'Nonprofit Organization Management', 'nan', 'nan', 'nan', 'nan', 'nan', 'Computer Software', 'Computer Software', 'Computer Software', 'Computer Software', 'Computer Software', 'Internet', 'Internet', 'Internet', 'Internet', 'Internet', 'Information Technology and Services', 'Information Technology and Services', 'Information Technology and Services', 'Information Technology and Services', 'Information Technology and Services', 'Civic Social Organization', 'Civic Social Organization', 'Civic Social Organization', 'Civic Social Organization', 'Civic S

In [18]:
experience['industry']= experience['industry'].apply(clean_industry)

In [19]:
experience['industry'].value_counts()

Computer Software                      55
Information Technology and Services    45
Civic Social Organization              35
Management Consulting                  20
nan                                    20
Higher Education                       20
Internet                               10
Venture Capital Private Equity          5
Nonprofit Organization Management       5
Graphic Design                          5
Performing Arts                         5
Marketing and Advertising               5
Research                                5
Name: industry, dtype: int64

In [20]:
experience.drop(columns='index', inplace=True)
experience.head()

,ind_id,locationName,companyName,description,title,startDate_month,startDate_year,endDate_month,endDate_year,industry,company_empl_low,company_empl_high,title_direction,titleconsulting,titlefounder
0,0,Paris Metropolitan Region,bluenove,Bluenove accompagne la transformation positive...,Directeur associé,11,2017,0,0,Management Consulting,11,50,1,0,0
1,0,Paris Metropolitan Region,démocratie ouverte,Démocratie Ouverte est un collectif citoyen in...,Co-Président,1,2018,5,2020,Management Consulting,11,50,1,0,0
2,0,Paris Metropolitan Region,apm - association progrès du management,NaN,Expert,1,2017,0,0,Management Consulting,11,50,0,0,0
3,0,NaN,démocratie ouverte,NaN,Membre du Comité d'Orientation Stratégique,7,2021,0,0,Management Consulting,11,50,0,0,0
4,0,Paris Metropolitan Region,dassault systèmes,Netvibes provides Dashboard Intelligence ~ tra...,"Senior Director, Strategic Business Development",2,2013,10,2017,Management Consulting,11,50,1,0,0


In [21]:
experience.reset_index(inplace=True)

In [22]:
experience.rename(columns={"index":"exp_id"}, inplace=True)

In [23]:
experience.dtypes

exp_id                int64
ind_id                int64
locationName         object
companyName          object
description          object
title                object
startDate_month       int64
startDate_year        int64
endDate_month         int64
endDate_year          int64
industry             object
company_empl_low      int64
company_empl_high     int64
title_direction       int64
titleconsulting       int64
titlefounder          int64
dtype: object

In [24]:
experience.columns

Index(['exp_id', 'ind_id', 'locationName', 'companyName', 'description',
       'title', 'startDate_month', 'startDate_year', 'endDate_month',
       'endDate_year', 'industry', 'company_empl_low', 'company_empl_high',
       'title_direction', 'titleconsulting', 'titlefounder'],
      dtype='object')

In [25]:
experience.head()

,exp_id,ind_id,locationName,companyName,description,title,startDate_month,startDate_year,endDate_month,endDate_year,industry,company_empl_low,company_empl_high,title_direction,titleconsulting,titlefounder
0,0,0,Paris Metropolitan Region,bluenove,Bluenove accompagne la transformation positive...,Directeur associé,11,2017,0,0,Management Consulting,11,50,1,0,0
1,1,0,Paris Metropolitan Region,démocratie ouverte,Démocratie Ouverte est un collectif citoyen in...,Co-Président,1,2018,5,2020,Management Consulting,11,50,1,0,0
2,2,0,Paris Metropolitan Region,apm - association progrès du management,NaN,Expert,1,2017,0,0,Management Consulting,11,50,0,0,0
3,3,0,NaN,démocratie ouverte,NaN,Membre du Comité d'Orientation Stratégique,7,2021,0,0,Management Consulting,11,50,0,0,0
4,4,0,Paris Metropolitan Region,dassault systèmes,Netvibes provides Dashboard Intelligence ~ tra...,"Senior Director, Strategic Business Development",2,2013,10,2017,Management Consulting,11,50,1,0,0


In [26]:
experience.to_csv("sql/experience.csv", sep=";", index=False)

In [27]:
experience.columns

Index(['exp_id', 'ind_id', 'locationName', 'companyName', 'description',
       'title', 'startDate_month', 'startDate_year', 'endDate_month',
       'endDate_year', 'industry', 'company_empl_low', 'company_empl_high',
       'title_direction', 'titleconsulting', 'titlefounder'],
      dtype='object')

In [28]:

experience.to_sql('people_experience',engine, 'civictech', if_exists='replace', index=False)# dtype={'locationName': string, 'companyName':'VARCHAR(500)', 'description':'VARCHAR(600)', 'title': 'VARCHAR(700)', 'industry':'VARCHAR(500)'})


235

### Education

In [29]:
education=pd.read_csv('profiles/people_education_db.csv', sep=';')

In [30]:
print(list(education.columns))
print(len(list(education.columns)))
print(len(education))

['ind_id', 'school', 'degreeName', 'schoolName', 'fieldOfStudy', 'startDate_year']
6
141


In [31]:
education.isna().sum().sort_values(ascending=False)

fieldOfStudy      51
school            46
degreeName        44
startDate_year    30
schoolName        19
ind_id             0
dtype: int64

In [32]:
education.dtypes.sort_values(ascending=False)

school             object
degreeName         object
schoolName         object
fieldOfStudy       object
startDate_year    float64
ind_id              int64
dtype: object

In [33]:
education.startDate_year.value_counts(dropna=False)
education['startDate_year'] = education['startDate_year'].fillna(0)

In [34]:
education['startDate_year']= education['startDate_year'].astype(int)

In [35]:
education.to_csv("sql/education_2.csv", sep=";", index=False)

In [36]:
education.to_sql('people_education', engine, 'civictech', if_exists='replace', index=False)

141

# Organizations

## Companies finance data

In [37]:
companies_finance=pd.read_csv('organizations/companies_finance_data.csv', sep=';') # possible index drop

In [38]:
companies_finance.head()

,company,annee,chiffre_daffaires_e,marge_brute_e,resultat_dexploitation_e,resultat_net_e,taux_croissance_ca_pc,taux_marge_brute_pc,taux_marge_operationnelle_pc,gestion_bfr,...,marge_nette_pc,rentabilite_sur_fonds_propres_pc,rentabilite_economique_pc,valeur_ajoutee_e,valeur_ajoutee_sur_ca_pc,structure_dactivite,salaires_et_charges_sociales_e,salaires_sur_ca_pc,impots_et_taxes_e,chiffre_daffaires_a_lexport_e
0,citility,2017.0,0.0,0,0,-562000,0,0,0,2017.0,...,0,0,0,0,0,2017.0,0,0,0,0
1,citility,2016.0,30700.0,527000,-296000,-238000,4,1720,-965,2016.0,...,-775,-623,-173,351000,1140,2016.0,636000,2070,9160,0
2,voxcracy,2019.0,46300.0,46300,-38500,-39000,177,100,-833,2019.0,...,-842,-76,-63,2170,47,2019.0,33600,725,137,0
3,voxcracy,2018.0,16700.0,157000,-130000,-112000,861,940,-776,2018.0,...,-668,-202,-169,-21000,-126,2018.0,101000,602,1120,5000
4,voxcracy,2017.0,1740.0,75900,-30300,-23400,0,4360,-1740,2017.0,...,-1350,-41,-39,-5040,-290,2017.0,19100,1100,329,0


In [39]:
companies_finance.drop(columns=['gestion_bfr', 'autonomie_financiere','solvabilite', 'rentabilite','structure_dactivite'], inplace=True)

In [40]:
companies_finance.reset_index(inplace=True)

In [41]:
companies_finance.rename(columns={"index": "company_year_ide"}, inplace=True)

In [42]:
companies_finance.head()

,company_year_ide,company,annee,chiffre_daffaires_e,marge_brute_e,resultat_dexploitation_e,resultat_net_e,taux_croissance_ca_pc,taux_marge_brute_pc,taux_marge_operationnelle_pc,...,fonds_propres_e,marge_nette_pc,rentabilite_sur_fonds_propres_pc,rentabilite_economique_pc,valeur_ajoutee_e,valeur_ajoutee_sur_ca_pc,salaires_et_charges_sociales_e,salaires_sur_ca_pc,impots_et_taxes_e,chiffre_daffaires_a_lexport_e
0,0,citility,2017.0,0.0,0,0,-562000,0,0,0,...,1380000,0,0,0,0,0,0,0,0,0
1,1,citility,2016.0,30700.0,527000,-296000,-238000,4,1720,-965,...,382000,-775,-623,-173,351000,1140,636000,2070,9160,0
2,2,voxcracy,2019.0,46300.0,46300,-38500,-39000,177,100,-833,...,513000,-842,-76,-63,2170,47,33600,725,137,0
3,3,voxcracy,2018.0,16700.0,157000,-130000,-112000,861,940,-776,...,552000,-668,-202,-169,-21000,-126,101000,602,1120,5000
4,4,voxcracy,2017.0,1740.0,75900,-30300,-23400,0,4360,-1740,...,569000,-1350,-41,-39,-5040,-290,19100,1100,329,0


In [43]:
companies_finance.to_csv("sql/companies_finance.csv", sep=";", index=False)

In [44]:
companies_finance.to_sql('companies_finance', engine, 'civictech', if_exists='replace', index=False)

74

## Organizations general data

### initial cleaning for all

In [45]:
companies_info=pd.read_csv('organizations/companies_info_data.csv', sep=';')
print(companies_info.columns)

Index(['company_id', 'companyName', 'Adresse :', 'Activité :', 'Effectif :',
       'Création :', 'Dirigeants :', 'Forme juridique :',
       'Inscription au RCS :', 'Capital social :',
       'Activité principale déclarée :', 'Code NAF ou APE :',
       'Domaine d’activité :', 'Dirigeant :', 'Inscription au RNA :',
       'Identifiant association :', 'Objet de l'association :',
       'Statut INSEE :'],
      dtype='object')


In [46]:
companies_info.head()

,company_id,companyName,Adresse :,Activité :,Effectif :,Création :,Dirigeants :,Forme juridique :,Inscription au RCS :,Capital social :,Activité principale déclarée :,Code NAF ou APE :,Domaine d’activité :,Dirigeant :,Inscription au RNA :,Identifiant association :,Objet de l'association :,Statut INSEE :
0,0,citility,5 RUE DE LA CLAIRE 69009 LYON 9EME,Édition de logiciels applicatifs,0 salarié (donnée 2019),05/05/2014,"André MAY, ODICEO, Sabine SCHNECK","SAS, société par actions simplifiée","INSCRIT (au greffe de LYON, le 12/05/2014)","39 620,00 €",Edition de logiciels applicatifs.,58.29C (Édition de logiciels applicatifs),Édition,NaN,NaN,NaN,NaN,NaN
1,1,poligma,RPT BENJAMIN FRANKLIN 34960 MONTPELLIER CEDEX 2,Conseil en systèmes et logiciels informatiques,Entre 3 et 5 salariés (donnée 2020),01/09/2015,NaN,"SAS, société par actions simplifiée","INSCRIT (au greffe de MONTPELLIER, le 11/09/2015)","20 944,00 €",Développement de services informatiques et num...,62.02A (Conseil en systèmes et logiciels infor...,"Programmation, conseil et autres activités inf...",Philippe GERARD,NaN,NaN,NaN,NaN
2,2,voxcracy,1133 RTE FENERIE 06580 PEGOMAS,Programmation informatique,0 salarié,01/09/2014,"Olivier ROCCA, Pascal RUSCICA","SAS, société par actions simplifiée","INSCRIT (au greffe de GRASSE, le 17/07/2014)","1 000,00 €","La recherche, le développement et la commercia...",62.01Z (Programmation informatique),"Programmation, conseil et autres activités inf...",NaN,NaN,NaN,NaN,NaN
3,3,LLL_2,16 RUE DU CAIRE 75002 PARIS 2,Autres activités de soutien aux entreprises n....,Entre 3 et 5 salariés (donnée 2020),16/01/2015,"ANNAMAMASHOW, Raymond Maeder, 3APEXCO","SAS, société par actions simplifiée","INSCRIT (au greffe de PARIS, le 03/02/2015)","45 000,00 €",Développement de nouvelles formes de collabora...,82.99Z (Autres activités de soutien aux entrep...,Activités administratives et autres activités ...,NaN,NaN,NaN,NaN,NaN
4,4,bluenove,112 B RUE CARDINET 75017 PARIS 17,Conseil pour les affaires et autres conseils d...,Entre 20 et 49 salariés (donnée 2020),23/01/2008,"GROUPE BLUENOVE INC., Guillaume Drancy, Carole...","SAS, société par actions simplifiée","INSCRIT (au greffe de PARIS, le 21/11/2019)","56 445,00 €",L'activité de conseil en stratégie d'accompagn...,70.22Z (Conseil pour les affaires et autres co...,Activités des sièges sociaux ; conseil de gestion,NaN,NaN,NaN,NaN,NaN


In [47]:
new_cols=[]
for i in list(companies_info.columns):
    i=i.lower().replace(' :', '').replace('é', 'e').replace("(", '').replace(')', '').replace(' ', '_').replace("'",'').replace("’",'').replace("è", "e")
    new_cols.append(i)
print(new_cols)

['company_id', 'companyname', 'adresse', 'activite', 'effectif', 'creation', 'dirigeants', 'forme_juridique', 'inscription_au_rcs', 'capital_social', 'activite_principale_declaree', 'code_naf_ou_ape', 'domaine_dactivite', 'dirigeant', 'inscription_au_rna', 'identifiant_association', 'objet_de_lassociation', 'statut_insee']


In [48]:
companies_info.columns=new_cols
companies_info.head()

,company_id,companyname,adresse,activite,effectif,creation,dirigeants,forme_juridique,inscription_au_rcs,capital_social,activite_principale_declaree,code_naf_ou_ape,domaine_dactivite,dirigeant,inscription_au_rna,identifiant_association,objet_de_lassociation,statut_insee
0,0,citility,5 RUE DE LA CLAIRE 69009 LYON 9EME,Édition de logiciels applicatifs,0 salarié (donnée 2019),05/05/2014,"André MAY, ODICEO, Sabine SCHNECK","SAS, société par actions simplifiée","INSCRIT (au greffe de LYON, le 12/05/2014)","39 620,00 €",Edition de logiciels applicatifs.,58.29C (Édition de logiciels applicatifs),Édition,NaN,NaN,NaN,NaN,NaN
1,1,poligma,RPT BENJAMIN FRANKLIN 34960 MONTPELLIER CEDEX 2,Conseil en systèmes et logiciels informatiques,Entre 3 et 5 salariés (donnée 2020),01/09/2015,NaN,"SAS, société par actions simplifiée","INSCRIT (au greffe de MONTPELLIER, le 11/09/2015)","20 944,00 €",Développement de services informatiques et num...,62.02A (Conseil en systèmes et logiciels infor...,"Programmation, conseil et autres activités inf...",Philippe GERARD,NaN,NaN,NaN,NaN
2,2,voxcracy,1133 RTE FENERIE 06580 PEGOMAS,Programmation informatique,0 salarié,01/09/2014,"Olivier ROCCA, Pascal RUSCICA","SAS, société par actions simplifiée","INSCRIT (au greffe de GRASSE, le 17/07/2014)","1 000,00 €","La recherche, le développement et la commercia...",62.01Z (Programmation informatique),"Programmation, conseil et autres activités inf...",NaN,NaN,NaN,NaN,NaN
3,3,LLL_2,16 RUE DU CAIRE 75002 PARIS 2,Autres activités de soutien aux entreprises n....,Entre 3 et 5 salariés (donnée 2020),16/01/2015,"ANNAMAMASHOW, Raymond Maeder, 3APEXCO","SAS, société par actions simplifiée","INSCRIT (au greffe de PARIS, le 03/02/2015)","45 000,00 €",Développement de nouvelles formes de collabora...,82.99Z (Autres activités de soutien aux entrep...,Activités administratives et autres activités ...,NaN,NaN,NaN,NaN,NaN
4,4,bluenove,112 B RUE CARDINET 75017 PARIS 17,Conseil pour les affaires et autres conseils d...,Entre 20 et 49 salariés (donnée 2020),23/01/2008,"GROUPE BLUENOVE INC., Guillaume Drancy, Carole...","SAS, société par actions simplifiée","INSCRIT (au greffe de PARIS, le 21/11/2019)","56 445,00 €",L'activité de conseil en stratégie d'accompagn...,70.22Z (Conseil pour les affaires et autres co...,Activités des sièges sociaux ; conseil de gestion,NaN,NaN,NaN,NaN,NaN


In [49]:
companies_info['creation'] = pd.to_datetime(companies_info['creation'], format="%d/%m/%Y")

### creating associations table

In [50]:
# to split companies and NGOs, just 
associations= pd.DataFrame(companies_info.loc[companies_info['activite']=="Autres organisations fonctionnant par adhésion volontaire"])

In [51]:
a= pd.DataFrame(associations.isna().sum().sort_values(ascending=False))
a.columns=["column"]
coltodrop= list(a.loc[a['column']>=15].index)
associations.drop(columns=coltodrop, inplace=True)
associations.drop(columns=['activite','forme_juridique', 'inscription_au_rcs', 'code_naf_ou_ape', 'domaine_dactivite', 'identifiant_association'], inplace=True)

In [52]:
associations.rename(columns={"company_id":"asso_id", "companyname":"association_name"}, inplace=True)

In [53]:
def clean_inscriptions(x):
    x=str(x)
    if "INSCRIT" in x:
        return 1
    else:
        return 0

In [54]:
associations["inscription_rna"]=associations['inscription_au_rna'].apply(clean_inscriptions)

In [55]:
def get_date(x):
    x=str(x)
    pattern=r"\d{1,5}/\d{2,5}/\d{2,5}"
    a= re.findall(pattern, x)
    a= ''.join(a).strip()
    return a

In [56]:
associations["date_inscr"]=pd.to_datetime(associations['inscription_au_rna'].apply(get_date), dayfirst=True)

In [57]:
associations.drop(columns="inscription_au_rna", inplace=True)

In [58]:
associations.head()

,asso_id,association_name,adresse,effectif,creation,objet_de_lassociation,inscription_rna,date_inscr
6,6,make4,14 RUE ST GUILLAUME 75007 PARIS 7,Entre 3 et 5 salariés (donnée 2020),2017-01-11,NaN,0,NaT
8,8,voteetvous,133 RUE ST DOMINIQUE 75007 PARIS 7,Au moins 1 salarié (donnée 2023),2014-01-27,"Renforcer l'exercice démocratique du vote, en ...",1,2014-01-27
26,26,lesbricodeurs,8 PL LOUIS CHAZETTE 69001 LYON 1ER,Entre 1 et 2 salariés (donnée 2020),2016-04-30,Diffuser la culture numérique et accompagner d...,1,2015-11-05
38,38,democracyos,17 RUE MYRHA 75018 PARIS 18,Au moins 1 salarié (donnée 2023),2015-04-21,Représenter la communauté qui développe et qui...,1,2015-04-21
39,39,polipart,70 BD DE CLICHY 75018 PARIS 18,0 salarié (donnée 2023),2019-03-09,"Améliorer la relation entre institutions, élus...",1,2019-03-09


### Companies table

In [59]:
companies= pd.DataFrame(companies_info.loc[companies_info['activite']!="Autres organisations fonctionnant par adhésion volontaire"])

In [60]:
companies.drop(columns=['inscription_au_rna', 'identifiant_association', 'objet_de_lassociation'], inplace=True)

In [61]:
companies.drop(columns=['statut_insee'], inplace=True)

In [62]:
#companies.inscription_au_rcs.value_counts()

In [63]:
companies["inscription_rcs"]=companies['inscription_au_rcs'].apply(clean_inscriptions)

In [64]:
import re

def greffe(x):
     x=str(x)
     pattern = " [A-Z]+"
     a= re.findall(pattern, x)
     a= ''.join(a).strip()
     return a

In [65]:
companies["greffe"]= companies['inscription_au_rcs'].apply(greffe)

In [66]:
companies["date_inscr_rad"]=pd.to_datetime(companies['inscription_au_rcs'].apply(get_date), dayfirst=True)

In [67]:
def select_appr_col(x, y): 
    x=str(x)
    y=str(y)
    if x=="NaN":
        return y
    elif y=="Nan":
        return x

In [68]:
companies["directors"]=companies["dirigeants"].astype(str).replace("nan", '')+companies["dirigeant"].astype(str).replace("nan", '')

In [69]:
companies.drop(columns=["dirigeants", "dirigeant", "inscription_au_rcs"], inplace=True)

In [70]:
import re
def fix_K_social(x):
    x=str(x)
    x.replace('€', '')
    x.replace(' ', '')
    if "," in x: 
        y= re.split(',', x)[0]
        y= re.split(' ', y)
        return "".join(y)
    else:
        return 0


In [71]:
#test line 
# companies.capital_social.apply(fix_K_social).astype(int)

In [72]:
companies["capital_social"]= companies.capital_social.apply(fix_K_social).astype(int)

In [73]:
print(list(companies.companyname))

['citility', 'poligma', 'voxcracy', 'LLL_2', 'bluenove', 'ecivis', 'civimetric', 'vooter', 'organigram_97', 'forcity_mere', 'mymairie', 'quorum', 'fullmobs', 'mairesetcitoyens', 'parlonspo', 'make3', 'consultvox', 'loomio', 'politiker', 'citizenlabbv', 'datagora', 'ouiville', 'forcity', 'accromedias', 'crowdpac', 'ledrenche', 'youvote', 'make2', 'ecreall_nova_ideo', 'neocity', 'citieszen', 'politicus', 'capcollectif', 'whip', 'voxestudio', 'LLL', 'citybay_ambitions', 'politizr', 'digitalebox', 'mesopinions', 'madeinvote', 'kawaa', 'ciwik', 'digiworks', 'equivote_myopencity', 'make', 'kohero', 'openagora', 'moneparti', 'nousrassemble', 'bittle', 'marston_massiet', 'koom', 'reperage_urbain', 'D21', 'IDcity', 'Stig', 'opensourcepolitics', 'GOV', 'fluicity', 'civocracy']


In [74]:
companies['companyname'] = companies['companyname'].replace(['LLL', 'make', 'citizenlabbv', 'voxestudio', 'marston_massiet'], ['liberté_living_lab', 'make.org', 'citizenlobbv', 'VOXE', 'marston'])

In [75]:
companies.head()

,company_id,companyname,adresse,activite,effectif,creation,forme_juridique,capital_social,activite_principale_declaree,code_naf_ou_ape,domaine_dactivite,inscription_rcs,greffe,date_inscr_rad,directors
0,0,citility,5 RUE DE LA CLAIRE 69009 LYON 9EME,Édition de logiciels applicatifs,0 salarié (donnée 2019),2014-05-05,"SAS, société par actions simplifiée",39620,Edition de logiciels applicatifs.,58.29C (Édition de logiciels applicatifs),Édition,1,LYON,2014-05-12,"André MAY, ODICEO, Sabine SCHNECK"
1,1,poligma,RPT BENJAMIN FRANKLIN 34960 MONTPELLIER CEDEX 2,Conseil en systèmes et logiciels informatiques,Entre 3 et 5 salariés (donnée 2020),2015-09-01,"SAS, société par actions simplifiée",20944,Développement de services informatiques et num...,62.02A (Conseil en systèmes et logiciels infor...,"Programmation, conseil et autres activités inf...",1,MONTPELLIER,2015-09-11,Philippe GERARD
2,2,voxcracy,1133 RTE FENERIE 06580 PEGOMAS,Programmation informatique,0 salarié,2014-09-01,"SAS, société par actions simplifiée",1000,"La recherche, le développement et la commercia...",62.01Z (Programmation informatique),"Programmation, conseil et autres activités inf...",1,GRASSE,2014-07-17,"Olivier ROCCA, Pascal RUSCICA"
3,3,LLL_2,16 RUE DU CAIRE 75002 PARIS 2,Autres activités de soutien aux entreprises n....,Entre 3 et 5 salariés (donnée 2020),2015-01-16,"SAS, société par actions simplifiée",45000,Développement de nouvelles formes de collabora...,82.99Z (Autres activités de soutien aux entrep...,Activités administratives et autres activités ...,1,PARIS,2015-02-03,"ANNAMAMASHOW, Raymond Maeder, 3APEXCO"
4,4,bluenove,112 B RUE CARDINET 75017 PARIS 17,Conseil pour les affaires et autres conseils d...,Entre 20 et 49 salariés (donnée 2020),2008-01-23,"SAS, société par actions simplifiée",56445,L'activité de conseil en stratégie d'accompagn...,70.22Z (Conseil pour les affaires et autres co...,Activités des sièges sociaux ; conseil de gestion,1,PARIS,2019-11-21,"GROUPE BLUENOVE INC., Guillaume Drancy, Carole..."


### export to sql and csv

In [76]:
companies.to_csv("sql/companies_info.csv", sep=";", index=False)

In [77]:
companies.to_sql('companies_info', engine, 'civictech', if_exists='replace', index=False)

61

In [78]:
associations.to_csv("sql/associations_info.csv", sep=';', index=False)

In [79]:
associations.to_sql("associations_info", engine, 'civictech', if_exists='replace', index=False)

15

In [80]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html

# you can pass data types here 
# from sqlalchemy.types import Integer
# df.to_sql('integers', con=engine, index=False,
#           dtype={"A": Integer()})